# Building Autonomous Agents to Create Analysis Reports
## Introduction

In this lesson, our aim is to create an autonomous agent using the LangChain framework. We will explore the concept of "Plan and Execute" LangChain agents and their ability to generate insightful analysis reports based on retrieved documents from Deep Lake.

We will start by understanding the fundamentals of the "Plan and Execute" LangChain agent framework and its benefits for complex long-term planning. Then, we will delve into our project's implementation details and workflow. 

By the end of the lesson, you will have a solid understanding of building autonomous agents using the LangChain framework and be equipped with the skills to create analysis reports using them.

### Workflow
This is the workflow we’ll follow in this project:

1. Saving Documents on Deep Lake:
We will begin by learning how to save documents on Deep Lake, which serves as our knowledge repository. Deep Lake provides information that our agents can leverage for analysis and report generation.
2. Creating a Document Retrieval Tool:
Next, we will develop a tool that enables our agent to retrieve the most relevant documents from Deep Lake based on a given query.
3. Using the Plan and Execute Agent:
The core of our project involves employing a "Plan and Execute" agent to devise a plan for answering a specific query about creating an overview of a topic. Our specific objective is to generate a comprehensive outline of recent events related to Artificial Intelligence regulations by governments, but the final agent could also work for other similar objectives as well.
To accomplish this, we will feed the query into the planner component of the agent, which will utilize a language model's reasoning ability to plan out the steps required. The planner will consider various factors, including the complexity of the query and instructions for the tool used to generate a step-by-step plan or lower-level queries.

The plan will then be passed to the executor component, which will determine the appropriate tools or actions required to execute each step of the plan. The executor, initially implemented as an Action Agent, will make use of the tools we developed earlier, such as the document retrieval tool, to gather relevant information and execute the plan.

## Plan and Execute

Plan and Execute agents are a new type of agent executor offering a different approach than the traditional agents supported in LangChain. These agents are heavily inspired by the BabyAGI framework and the recent Plan-and-Solve paper. The primary goal of "Plan and Execute" agents is to enable more complex long-term planning, even at the cost of making more calls to the language model.

- The planner in the "Plan-and-Execute" framework typically utilizes a language model's reasoning ability to plan out steps and handle ambiguity and edge cases.
- The executor, initially an Action Agent, takes the planner's high-level objectives (steps) and determines the tools or actions required to accomplish each step.

This separation of planning and execution allows for improved reliability and flexibility. It also facilitates the possibility of replacing these components with smaller, fine-tuned models in the future.

## Implementation

We then use the requests library to send HTTP requests and the newspaper library for article parsing. By iterating over a list of article URLs, the code downloads the HTML of each webpage, extracts the article text, and stores it along with the corresponding URL. We could also load our private files on Deep Lake, but for this project's scope, we’ll upload content downloaded from public web pages. 

In [2]:
import os

#os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"
#os.environ["ACTIVELOOP_TOKEN"] = "<YOUR-ACTIVELOOP-TOKEN>"
# We scrape several Artificial Intelligence news

import requests
from newspaper import Article # https://github.com/codelucas/newspaper
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_urls = [
    "https://www.artificialintelligence-news.com/2023/05/23/meta-open-source-speech-ai-models-support-over-1100-languages/",
    "https://www.artificialintelligence-news.com/2023/05/18/beijing-launches-campaign-against-ai-generated-misinformation/"
    "https://www.artificialintelligence-news.com/2023/05/16/openai-ceo-ai-regulation-is-essential/",
    "https://www.artificialintelligence-news.com/2023/05/15/jay-migliaccio-ibm-watson-on-leveraging-ai-to-improve-productivity/",
    "https://www.artificialintelligence-news.com/2023/05/15/iurii-milovanov-softserve-how-ai-ml-is-helping-boost-innovation-and-personalisation/",
    "https://www.artificialintelligence-news.com/2023/05/11/ai-and-big-data-expo-north-america-begins-in-less-than-one-week/",
    "https://www.artificialintelligence-news.com/2023/05/11/eu-committees-green-light-ai-act/",
    "https://www.artificialintelligence-news.com/2023/05/09/wozniak-warns-ai-will-power-next-gen-scams/",
    "https://www.artificialintelligence-news.com/2023/05/09/infocepts-ceo-shashank-garg-on-the-da-market-shifts-and-impact-of-ai-on-data-analytics/",
    "https://www.artificialintelligence-news.com/2023/05/02/ai-godfather-warns-dangers-and-quits-google/",
    "https://www.artificialintelligence-news.com/2023/04/28/palantir-demos-how-ai-can-used-military/",
    "https://www.artificialintelligence-news.com/2023/04/26/ftc-chairwoman-no-ai-exemption-to-existing-laws/",
    "https://www.artificialintelligence-news.com/2023/04/24/bill-gates-ai-teaching-kids-literacy-within-18-months/",
    "https://www.artificialintelligence-news.com/2023/04/21/google-creates-new-ai-division-to-challenge-openai/"
]

session = requests.Session()
pages_content = [] # where we save the scraped articles

for url in article_urls:
    try:
        time.sleep(2) # sleep two seconds for gentle scraping
        response = session.get(url, headers=headers, timeout=10)

        if response.status_code == 200:
            article = Article(url)
            article.download() # download HTML of webpage
            article.parse() # parse HTML to extract the article text
            pages_content.append({ "url": url, "text": article.text })
        else:
            print(f"Failed to fetch article at {url}")
    except Exception as e:
        print(f"Error occurred while fetching article at {url}: {e}")

#If an error occurs while fetching an article, we catch the exception and print
#an error message. This ensures that even if one article fails to download,
#the rest of the articles can still be processed.


Then, we import the OpenAIEmbeddings class, which will be used to compute embeddings for our documents. We also import the Deep Lake class from the langchain.vectorstores module will serve as the storage for our documents and their embeddings. 

In [3]:
# We'll use an embedding model to compute our documents' embeddings
from langchain.embeddings.openai import OpenAIEmbeddings

# We'll store the documents and their embeddings in the deep lake vector db
from langchain.vectorstores import DeepLake

# Setup deep lake
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "edumunozsala"
my_activeloop_dataset_name = "langchain_course_analysis_outline"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Could not import azure.core python package.
c:\Users\edumu\anaconda3\envs\llm-lc\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.3) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


Your Deep Lake dataset has been successfully created!
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/edumunozsala/langchain_course_analysis_outline
hub://edumunozsala/langchain_course_analysis_outline loaded successfully.


Next, we create an instance of RecursiveCharacterTextSplitter with specified chunk_size and chunk_overlap parameters. Then, we iterated over the pages_content and use the split_text method of the text_splitter to split each article text into chunks. These chunks are then appended to the all_texts list, resulting in a collection of smaller text chunks derived from the original articles. 

In [4]:
# We split the article texts into small chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

all_texts = []
for d in pages_content:
    chunks = text_splitter.split_text(d["text"])
    for chunk in chunks:
        all_texts.append(chunk)

In [5]:
# we add all the chunks to the Deep lake
db.add_texts(all_texts)

Evaluating ingest: 100%|██████████| 1/1 [00:17<00:00


Dataset(path='hub://edumunozsala/langchain_course_analysis_outline', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape      dtype  compression
  -------   -------   -------    -------  ------- 
 embedding  generic  (94, 1536)  float32   None   
    ids      text     (94, 1)      str     None   
 metadata    json     (94, 1)      str     None   
   text      text     (94, 1)      str     None   


['22157702-75b4-11ee-835e-cc2f714963ed',
 '22159df2-75b4-11ee-bb85-cc2f714963ed',
 '22159df3-75b4-11ee-ab94-cc2f714963ed',
 '22159df4-75b4-11ee-a19f-cc2f714963ed',
 '22159df5-75b4-11ee-a885-cc2f714963ed',
 '22159df6-75b4-11ee-af8f-cc2f714963ed',
 '22159df7-75b4-11ee-bfa6-cc2f714963ed',
 '22159df8-75b4-11ee-bf7b-cc2f714963ed',
 '22159df9-75b4-11ee-9913-cc2f714963ed',
 '22159dfa-75b4-11ee-b366-cc2f714963ed',
 '22159dfb-75b4-11ee-8b91-cc2f714963ed',
 '22159dfc-75b4-11ee-a575-cc2f714963ed',
 '22159dfd-75b4-11ee-8b87-cc2f714963ed',
 '22159dfe-75b4-11ee-b63f-cc2f714963ed',
 '22159dff-75b4-11ee-9012-cc2f714963ed',
 '22159e00-75b4-11ee-a765-cc2f714963ed',
 '22159e01-75b4-11ee-bff9-cc2f714963ed',
 '22159e02-75b4-11ee-8436-cc2f714963ed',
 '22159e03-75b4-11ee-aa6a-cc2f714963ed',
 '22159e04-75b4-11ee-bcc3-cc2f714963ed',
 '22159e05-75b4-11ee-8fa8-cc2f714963ed',
 '22159e06-75b4-11ee-b05a-cc2f714963ed',
 '22159e07-75b4-11ee-ac2d-cc2f714963ed',
 '22159e08-75b4-11ee-8677-cc2f714963ed',
 '22159e09-75b4-

 We are done with setting up the Deep Lake dataset with our documents! Let’s now focus on building the “Plan and Execute” agent that will leverage our dataset. Now, we can set up our Plan and Execute agent. Let’s create a retriever from the Deep Lake dataset and a function for our custom tool that retrieves the most similar documents to a query from the dataset.

In [6]:
# Get the retriever object from the deep lake db object and set the number
# of retrieved documents to 3
retriever = db.as_retriever()
retriever.search_kwargs['k'] = 3

# We define some variables that will be used inside our custom tool
CUSTOM_TOOL_DOCS_SEPARATOR ="\n---------------\n" # how to join together the retrieved docs to form a single string

# This is the function that defines our custom tool that retrieves relevant
# docs from Deep Lake
def retrieve_n_docs_tool(query: str) -> str:
    """Searches for relevant documents that may contain the answer to the query."""
    docs = retriever.get_relevant_documents(query)
    texts = [doc.page_content for doc in docs]
    texts_merged = "---------------\n" + CUSTOM_TOOL_DOCS_SEPARATOR.join(texts) + "\n---------------"
    return texts_merged

This functionality enables the plan and execution agent to retrieve and process relevant documents for further analysis and decision-making.

In [7]:
from langchain.agents.tools import Tool

# We create the tool that uses the "retrieve_n_docs_tool" function
tools = [
    Tool(
        name="Search Private Docs",
        func=retrieve_n_docs_tool,
        description="useful for when you need to answer questions about current events about Artificial Intelligence"
    )
]


The tool is named "Search Private Docs," and its functionality is based on the retrieve_n_docs_tool function. The purpose of this tool is to provide a way to search for and retrieve relevant documents from Deep Lake in order to answer questions about current events related to Artificial Intelligence.

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

# let's create the Plan and Execute agent
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)


The agent consists of two components: a planner and an executor. The planner is responsible for generating a plan based on the given input, and the executor executes the plan by interacting with the tools and external systems. The agent is set to be verbose, which means it will provide detailed information and logs during its operation.


In [9]:
# we test the agent
response = agent.run("Write an overview of Artificial Intelligence regulations by governments by country")



> Entering new PlanAndExecute chain...
steps=[Step(value='Research and gather information on the current state of Artificial Intelligence (AI) regulations by governments in different countries.'), Step(value='Organize the information by country and categorize it based on the level of AI regulation.'), Step(value='Provide an overview of the AI regulations in each country, including any specific laws or policies that have been implemented.'), Step(value='Include information on the key areas covered by the regulations, such as data privacy, algorithmic transparency, liability, and ethical considerations.'), Step(value='Highlight any notable differences or similarities between the regulations in different countries.'), Step(value='Summarize the overall trends and developments in AI regulations globally.'), Step(value='Given the above steps taken, provide an overview of Artificial Intelligence regulations by governments by country. ')]

> Entering new AgentExecutor chain...
Action:
```
{


At first, the planning agent creates a plan for our query with multiple steps. Each step is a query that the action agent will be asked to give an answer to. Here are the identified steps.

- Research the current state of Artificial Intelligence (AI) regulations in various countries.
- Identify the key countries with significant AI regulations or ongoing discussions about AI regulations.
- Summarize the AI regulations or discussions in each identified country.
- Organize the information by country, providing an overview of each AI regulation.
- Given the above steps taken, provide an overview of Artificial Intelligence regulations by governments by country.